# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 10
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus Polling - FIFO
- der Algorithmus weist den BEVs jeweils ein Zustand zu, die Zustände sind wartend (BEV parkt und will laden), ladend (BEV wird mit Solarenergie geladen), nicht parkend (BEV befindet sich nicht im Lademanagmentsystem)
- pro BEV Ladeleistung von z.B. 2kW (ladeleistung_pro_bev), wenn verfügbarer Solarstrom > Ladeleistung, dann wird der Solarstrom auf mehrere BEVs verteilt
- aktuell wartende Verbraucher werden in einer Warteliste (Pollingliste) abgearbeitet, nach einem variablen Zeitintervall (maximale_ladezeit, z.B. 30min) wird Verbraucher wieder hinten an die Warteliste angefügt. 
- dabei wird immer das erste BEV der Warteliste auf die Ladeliste genommen, unabhängig von der Länge der Parkzeit
- Wenn im Tagesverlauf die Anzahl bereits ladender BEVs größer wird als die Anzahl möglich ladender BEVs, wird das erste BEV der Ladeliste wieder nach hinten an die Warteliste gesetzt

### Verteilungsalgorithmus Polling - mit gleichmäßiger Verteilung
- wie FIFO Abarbeitung der BEVs auf der Warteliste
- allerdings werden BEVs nach dem sie einmal aufgeladen wurden auf eine neue Warteliste gesetzt mit geringerer Priorität => so soll gewährleistet werden, dass erstmal alle BEVs einmal aufgeladen werden, bevor BEVs mehrmals geladen werden 

### Prognose (Berücksichtigung der Parkdauer - Optimale Ausnutzung)
- Ziel ist das möglichst alle gleich viel bekommen und der Solarstrom optimal ausgenutzt wird ⇒ nichts geht verloren
- Berechnung des Ladeintervalls:
    - Wann sind während des angegebenen Parkintervalls am wenigsten andere BEVs am Warten? (Minimum Anzahl wartender BEVs in Parkintevall)
    - Ladestart: sobald Minimum Anzahl wartender BEVs in Parkintevall erreicht wird
    - Ladeende: nach fester Zeit oder nachdem Anzahl wartender BEVs nicht mehr Minimum Anzahl wartender BEVs entspricht
- Solarleistung wird zu jedem Zeitpunkt gleichmäßig unter ladenden BEVs verteilt, also Ladeleistung = Verfügbare Solarleistung / Ladende BEVs

### Prognose (Berücksichtigung der Parkdauer - Belohnung)
- Ziel: länger parkende BEVs sollen belohnt werden und mehr Solarenergie garantiert bekommen
- Möglichkeiten: mehr Solarenergie oder längeres Ladeintervall

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm)

distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
                    bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

{0: [(13.0, 7.3), 'wartend', []], 1: [(8.5, 4.3), 'nicht parkend', []], 2: [(8.5, 5.6), 'nicht parkend', []], 3: [(12.5, 4.8), 'wartend', []], 4: [(13.0, 4.1), 'wartend', []], 5: [(9.0, 5.0), 'nicht parkend', []], 6: [(8.5, 4.9), 'nicht parkend', []], 7: [(8.0, 7.7), 'nicht parkend', []], 8: [(13.0, 6.0), 'wartend', []], 9: [(8.5, 5.0), 'nicht parkend', []]} Start: BEVs_dict
für BEV mit ID 0
{480: -2.0, 495: 1.38, 510: 1.2, 525: 2.38, 540: 3.17, 555: 4.56, 570: 6.16, 585: 7.6, 600: 8.5, 615: 8.87, 630: 8.7, 645: 8.39, 660: 8.33, 675: 8.52, 690: 8.96, 705: 9.3, 720: 9.17, 735: 8.58, 750: 6.45, 765: 5.58, 780: 4.0, 795: 4.01, 810: 5.6, 825: 6.03, 840: 7.0, 855: 7.7, 870: 6.35, 885: 4.82, 900: 3.6, 915: 2.69, 930: 2.08, 945: 2.24, 960: 2.25} available_solar_power_per_bev_per_minute_dict
[780] parking_interval_in_minutes_as_list
{780: 4.0} available_solar_power_per_bev_in_parking_interval_dict
0.06666666666666667 Fairer Anteil für BEV


TypeError: get_parking_interval_in_minutes_as_list() missing 1 required positional argument: 'minute_interval'

### Ergebnis Algorithmus

In [ ]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [ ]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict)

In [ ]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day)

In [ ]:
figureGeneration.create_bev_number_figure(simulation_data)